## Recursive Least Squares : Variable Directional Forgetting 

- This notebook contains the implementation of the paper [Recursive Least Squares with Variable-Direction Forgetting -- Compensating for the loss of persistency](https://arxiv.org/abs/2003.03523)
- Generated a synthetic dataset with two parameters (beta, gamma) for test the algorithm convergence [Present as example_data_gen() method in the RLS_VDF class
- The algorithm is stable over increase in time steps
- Estimated parameters are quite close to the ones used for generation of the synthetic dataset

### Use cases

- Adaptive filtering: Enables adaptive filters to adapt to changing conditions by updating parameters based on relevant and recent information while forgetting outdated data

- Signal processing: Widely used in noise cancellation, echo cancellation, equalization, and channel estimation to accurately estimate and track time-varying parameters

- Non-stationary environments: Enhances adaptability in scenarios where input signal properties change over time or exhibit non-stationary behavior

- System identification: Useful for estimating parameters of unknown systems based on observed input-output data, particularly for tracking time-varying parameters and capturing changes in system dynamics

- Robustness to outliers: Improves robustness by adaptively adjusting the forgetting factor to assign less weight to outliers, reducing their impact on parameter estimation

- Online learning: Suitable for real-time learning scenarios where data arrives sequentially, allowing continuous adaptation and utilization of new information

- Resource optimization: Optimizes computational resources by allocating effort based on parameter adaptability, leading to improved efficiency in memory usage and computational complexity

In [1]:
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [2]:
# Λk is a positive definite (and thus symmetric) matrix
# theta_0 : initial estimate of theta


class RLS_VDF():
    def __init__(self,n_samples,lamda , e,parameters =2):
        self.parameters = parameters
        self.theta_0 = np.random.uniform(-1, 1, self.parameters)
        self.theta =  self.theta_0#The vector of unknown parameters   
        #replacing the scalar forgetting factor m by a data-dependent forgetting matrix Λ
        self.lamda = lamda
        self.e = e
        self.P = np.eye(self.parameters)
        self.A = np.linalg.inv(np.eye(self.parameters,self.parameters))
        self.theta_vals = []
        self.n_samples = n_samples
        self.pkcondition_number = []
        self.losses = []
        
        
    def example_data_gen(self, beta,gamma):
        '''Used for generating time series data for Example 1 [Equation 6] '''
        phi =np.zeros((self.n_samples, 2))
        y = np.array([0.00]*self.n_samples)
        x= []
        x.append(0.12)
        for i in range(1,self.n_samples) :
                v=x[i-1] +x[i-1]*(beta*(1-x[i-1]) - gamma) + 8e-5*np.random.normal(0, 1)
                x.append(v)
        x = np.array(x).reshape(self.n_samples,1)
        for i in range(self.n_samples-1) :
            phi[i][0] = (1 - x[i][0]) * x[i][0]
            phi[i][1] = -x[i][0]
            y[i]= (x[i+1][0] - x[i][0]) 

        return phi,y,x
        
    
    def update(self,phi,y):
        for i in range(self.n_samples-1):
            U, Sigma, VT = np.linalg.svd(self.P)
            psi_k = phi @ U
            column_norms = np.linalg.norm(psi_k, axis=0)
            lamda_bar = np.sqrt(self.lamda) * np.eye(self.parameters,self.parameters)
            for i in range(len(column_norms)):
                if column_norms[i] <= self.e: 
                    lamda_bar[i, i] = 1 
            lamda_k = U@lamda_bar@U.T   
            
            pbar_k = np.linalg.inv(lamda_k)@self.P@np.linalg.inv(lamda_k)
            self.P = (pbar_k - pbar_k@(phi.T)@(np.linalg.inv(np.eye(phi.shape[0]) + phi @ pbar_k @ phi.T))@phi@pbar_k)
            self.theta = self.theta + self.P@(phi.T)@(y - phi@self.theta) + self.P@((lamda_k@self.A@lamda_k - self.A)@(self.theta_0 - self.theta))
            self.A = lamda_k@self.A@lamda_k + phi.T@phi
            self.theta_vals.append(self.theta)
            e = y - phi @ self.theta
                    
            loss = np.linalg.norm(e)
            
            self.losses.append(loss)
            
            self.pkcondition_number.append(np.linalg.cond(self.P))
       
    
        return self.theta_vals , self.P, self.pkcondition_number ,self.losses
    
    def fit(self,phi,y):
        return self.update(phi,y)
            

    def plot_parameters(self):
        ''' After model training, it can be used to plot the parameters against time'''
        b_vals  = [list(i)[0] for i in self.theta_vals]
        g_vals  = [list(i)[1] for i in self.theta_vals]
        if len(b_vals) == self.n_samples-1 :
                data = [
                    go.Scatter(x=list(range(len(b_vals))),y=b_vals,mode='lines',name = 'Beta Values : Infection Rate'),
                    go.Scatter(x=list(range(len(g_vals))),y=g_vals,mode='lines',name  = 'Gamma Values : Recovery Rate')]

                layout = go.Layout(title='Infection and Recovery Rates over time (RLS- vdf)',xaxis=dict(title='Time step'),yaxis=dict(title='Value'))
                fig = go.Figure(data=data, layout=layout)
                fig.update_layout( plot_bgcolor='rgba(226, 237, 223, 90)')
                         
                fig.show()
        else : 
            print('Error : The model is not yet trained on the time series data')

In [3]:
def loss_plot(model, loss, time_steps=359, data='Epidemic Data'):
    fig = go.Figure()

    #layout
    fig.update_layout(
        title=f'Least Square Error vs Time Steps for {model} on {data}',
        xaxis=dict(title='Time steps'),
        yaxis=dict(title='Least Squares Error'),
        template='plotly_white'  # Adjust template as needed
    )

    # Adding trace for the loss plot
    fig.add_trace(go.Scatter(x=list(range(time_steps)), y=loss, mode='lines'))

    fig.show()
    del fig

In [4]:
rls_vdf = RLS_VDF(360,0.99991,1e-5)
phi_1,y_1,x_1 = rls_vdf.example_data_gen(0.65,0.35)
theta_valss_1, P_k, cond_pk_3, loss= rls_vdf.fit(phi_1,y_1)

print(f'Estimated Beta (RLS-vdf): {theta_valss_1[-1][0]}\nEstimated Gamma (RLS-vdf): {theta_valss_1[-1][1]}\n')
loss_plot('RLS-VDF', loss,359,'Epidemic Data')
rls_vdf.plot_parameters()

Estimated Beta (RLS-vdf): 0.6379599937328685
Estimated Gamma (RLS-vdf): 0.3391435780511553



## Observation 
#### - L2 Norm decreases with time
#### - Parameters converge near to the true values